In [1]:
# Simple Jupyter Notebook for normalizing DICOM Study Description
#
# Author: Paulo Kuriki

In [2]:
import numpy as np
import pandas as pd

In [3]:
# defining constants
GAD = ['POS', '+C', 'GD', 'GAD', ' CE', '_CE']
GAD_EXCLUSION = ['SEM', '/S', 'S/', 'PRE']
T1 = ['T1']
T2 = ['T2']
FLAIR = ['FLAIR']
SWI = ['SWI', 'SWAN', 'T2 GRE', 'T2*']
FIESTA = ['FIESTA', 'CISS', 'BALANCE', 'DRIVE']
TOF = ['TOF']
DWI = ['DIF', 'DWI', 'TRACE']
DWI_EXCLUSION = ['ADC', 'APP', 'EXP']
MPR = ['MPR']
ANGIO = ['VEN']
PRIMARY = ['ORIGINAL', 'PRIMARY']
SAG = ['SAG']
AXI = ['AX']
COR = ['COR']
FATSAT = ['STIR', 'FS', 'FAT', 'SAT']

In [4]:
def search_words_in_series(series_description: str, search_words: list, exclusion_words: list = []) -> bool:

    if series_description is np.nan:
        return False
    
    try:
        search_flag = False
        for word in search_words:
            if word.upper() in series_description.upper():
                search_flag = True
                break
    except Exception as e:
        print(f"Error searching for inclusion list {search_words} in the series description {series_description}")
        return False

    try:
        exclusion_flag = False
        for word in exclusion_words:
            if word.upper() in series_description.upper():
                exclusion_flag = True
                break
    except Exception as e:
        print(f"Error searching for exclusion list {exclusion_words} in the series description {series_description}")
        return  False

    found = search_flag and exclusion_flag is False

    return found

In [5]:
def IOP_Plane(IOP: list) -> str:
    """
    This function takes IOP of an image and returns its plane (Sagittal, Coronal, Transverse)
    ['1', '0', '0', '0', '0', '-1'] you are dealing with Coronal plane view
    ['0', '1', '0', '0', '0', '-1'] you are dealing with Sagittal plane view
    ['1', '0', '0', '0', '1', '0'] you are dealing with Axial plane view
    
    Useful when you would like to discover the plane based on the DICOM acquisition
    """

    try:
        IOP_round = [round(x) for x in IOP]
        plane = np.cross(IOP_round[0:3], IOP_round[3:6])
        plane = [abs(x) for x in plane]
        if plane[0] == 1:
            return "SAG"
        elif plane[1] == 1:
            return "COR"
        elif plane[2] == 1:
            return "AXI"
        else:
            return "UNK"
    except:
        return "UNK"

In [6]:
def normalize_series_description(series_description: str):
    
    # searching for image plane in series descriptions
    if search_words_in_series(series_description, AXI):
        plane = 'AXI'
    elif search_words_in_series(series_description, SAG):
        plane = 'SAG'
    elif search_words_in_series(series_description, COR):
        plane = 'COR'
    else:
        plane = 'UNK'
    
    # searching for image weight in series descriptions
    if search_words_in_series(series_description, FATSAT, T1):
        weight = 'STIR'
    elif search_words_in_series(series_description, FATSAT, T2):
        weight = 'T1 FS'
    elif search_words_in_series(series_description, FLAIR, T1):
        weight = 'FLAIR'
    elif search_words_in_series(series_description, SWI):
        weight = 'SWI'
    elif search_words_in_series(series_description, FIESTA):
        weight = 'FIESTA'
    elif search_words_in_series(series_description, T2, T1):
        weight = 'T2'
    elif search_words_in_series(series_description, T1, T2):
        weight = 'T1'
    else:
        weight = 'UNK'
        
    # searching for constrast in series descriptions
    if search_words_in_series(series_description, GAD):
        gad = ' GD'    
    else:
        gad = ''
    
    
    return f'{plane} {weight}{gad}'    
    

In [7]:
# simples use cases

# 1. calling as a simple function
normalize_series_description('COR T1 FAT GAD')

'COR T1 FS GD'

In [ ]:
# 2. creating a new normalized series description column in your pandas DataFrame
# 
df['norm_series'] = [normalize_series_description(x) for x in df['description']]